In [1]:
import random

def rosalia(nombre):
    frase = random.choice(["tra tra","malamente","con altura"])
    return f"{nombre} dice {frase}"

In [2]:
rosalia("Juan")

'Juan dice malamente'

In [3]:
def conGenteRandom(fn):
    def wrapper(*listaNombres):
        return ','.join([fn(n) for n in listaNombres])
    return wrapper

In [4]:
@conGenteRandom
def rosalia2(nombre):
    frase = random.choice(["tra tra","malamente","con altura"])
    return f"{nombre} dice {frase}"

In [5]:
rosalia2("Juan","Pedro","Marta")

'Juan dice malamente,Pedro dice tra tra,Marta dice tra tra'

In [6]:
@conGenteRandom
def pitbull(nombre):
    frase = random.choice(["Mr Worldwide","1,2,3","Dale"])
    return f"{nombre} {frase}"

pitbull("Juan","Pepe")

'Juan Dale,Pepe Dale'

In [7]:
def noOp(fn):
    return wrapper

@noOp
def suma(a,b):
    return a+b


suma(5,6)

NameError: name 'wrapper' is not defined

In [ ]:
debugEnabled = True
def debug(fn):
    def wrapper(*args,**kwargs):
        if debugEnabled:
            print(f"args={args} kwargs={kwargs}")
        ret = fn(*args,**kwargs)
        if debugEnabled:
            print(f"return={ret}")
        return ret
    return wrapper

In [8]:
@debug
def suma(a,b):
    return a+b

debugEnabled = True
suma(5,suma(8,suma(10,5)))

NameError: name 'debug' is not defined

In [9]:
debugEnabled = False
suma(5,suma(8,suma(10,5)))

NameError: name 'suma' is not defined

In [10]:
def checkPositionals(*positionalTypes):
    def decorate(fn):
        def wrapper(*args,**kwargs):
            typedCheck = [type(a)==b for a,b in zip(args,positionalTypes)]
            if all(typedCheck):
                return fn(*args,**kwargs)
            else:
                raise TypeError(f"Check the call to function, invalid types. {positionalTypes}")
        return wrapper
    return decorate

@checkPositionals(str)
def saluda(nombre):
    return f"Hola {nombre}"


In [11]:
saluda("pepe")

'Hola pepe'

In [12]:
saluda(3)

TypeError: Check the call to function, invalid types. (<class 'str'>,)

In [13]:
@checkPositionals(int,int)
def suma(a,b):
    return a+b

In [14]:
suma(5,7)

12

In [15]:
suma("hola",8)

TypeError: Check the call to function, invalid types. (<class 'int'>, <class 'int'>)

In [16]:
withZip = lambda fn: lambda *posi: fn(zip(*posi))

In [17]:
@withZip
def nombres(listanombres):
    return [f"Hola {nombre} {apellido}" for nombre, apellido in  listanombres]

In [18]:
nombres(["Juan","Marta","Ana"],["Garcia"]*3)

['Hola Juan Garcia', 'Hola Marta Garcia', 'Hola Ana Garcia']

In [19]:
["Garcia"]*3

['Garcia', 'Garcia', 'Garcia']

In [20]:
@debug
@checkPositionals(int,int)
def suma(a,b):
    return a+b


debugEnabled = False
suma(2,4)

NameError: name 'debug' is not defined

# Error handling with sentry and decorators

In [32]:
import os
import dotenv
dotenv.load_dotenv()
import sentry_sdk
from sentry_sdk import capture_exception

SENTRY_DSN=os.getenv("SENTRY_DSN")
sentry_sdk.init(SENTRY_DSN)

In [24]:
try:
    a = 8/0
except Exception as e:
    print(e)
    capture_exception(e)

division by zero


In [26]:
def captureError(fn):
    def wrapper(*args,**kwargs):
        try:
            fn(*args,**kwargs)
        except Exception as e:
            print(e)
            capture_exception(e)
            raise(e)
    return wrapper

In [31]:
@captureError
def main():
    raise ValueError("MAL")
    
    
main()

MAL


ValueError: MAL

# Decorador remember

In [43]:
previous = dict()

def remember(fn):
    def wrapper(*args):
        if args in previous:
            print("Using precalculated value")
            return previous[args]
        ret = fn(*args)
        previous[args] = ret
        return ret
    return wrapper

In [44]:
@remember
def calculaNota(*args):
    print("Calculando nota....")
    return sum([e**2 for e in args])

print(previous)
calculaNota(1,2,3,4,5,6)
print(previous)
calculaNota(1,2,3,4,5,6)

{}
Calculando nota....
{(1, 2, 3, 4, 5, 6): 91}
Using precalculated value


91

In [45]:
calculaNota(44,55,66)

Calculando nota....


9317

In [46]:
print(previous)


{(1, 2, 3, 4, 5, 6): 91, (44, 55, 66): 9317}


In [47]:
calculaNota(44,55,66)

Using precalculated value


9317

wrapper
